In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import git

#get the Git root location
git_repo = git.Repo('.', search_parent_directories=True)

#Load the data
df = pd.read_csv(f"{git_repo.working_tree_dir}\\Database\\data.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

#Get the data features values and sample labels
X = df.iloc[:, 1:].values
y = df['CLASS']

#Split the data in train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, 
                                                                    shuffle=True, stratify=y)

#Set the k-fold object
kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

#Parameters to be tested
parameters = {
    'C': [0.1, 1, 5, 10],
    'kernel': ['rbf', 'poly'],
    'gamma': [1, 0.6, 0.2, 0.1, 0.01, 0.001],
    'degree': [1, 2, 3],
    'coef0': [0, 0.1, 0.5, 1],
    'decision_function_shape': ['ovo', 'ovr']
}

#Evaluation metrics
scores = ['accuracy', 'f1_macro']

#Use of the GridSearchCV to assesses the best parameter combination
#The primary metric are based on the f1_score.
#In this example, the SVC() model is being utilized, but could be any classifiers.
grid_search = GridSearchCV(estimator = SVC(),
                           param_grid = parameters,
                           cv=kfold,
                           scoring = scores,
                           refit='f1_macro',
                           return_train_score = True,
                           n_jobs=-1).fit(x_train, y_train)

#Dataframe with the results
df_score = pd.DataFrame(grid_search.cv_results_)

#Dataframe sorted from the highest f1_score to the lower f1_score
df_score_sorted = df_score.sort_values(by='mean_test_f1_macro', ascending=False)

df_score_sorted.to_csv(f"{git_repo.working_tree_dir}\\GridSearchCV\\GridSearchCV_results.csv")